In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os, glob

### I/O

In [2]:
# raw data from model runs
runs_dir = 'M:/Data/Urban/BAUS/visualization_design/data/data_raw_runs'
run182 = os.path.join(runs_dir, 'run182')
run262 = os.path.join(runs_dir, 'run262')
run314 = os.path.join(runs_dir, 'run314')
run374 = os.path.join(runs_dir, 'run374')
run5 = os.path.join(runs_dir, 'run5')

# data ready for visualization
viz_dir = 'M:/Data/Urban/BAUS/visualization_design/data/data_viz_ready'
viz_csv_dir = os.path.join(viz_dir, 'csv')
viz_spatial_dir = os.path.join(viz_dir, 'spatial')
viz_crosswalks_dir = os.path.join(viz_dir, 'crosswalks')

# viz_182 = os.path.join(viz_dir, 'run182')
# viz_268 = os.path.join(viz_dir, 'run268')
# viz_314 = os.path.join(viz_dir, 'run314')
# viz_374 = os.path.join(viz_dir, 'run374')

### create growth geography spatial files

In [3]:
# parcels_geography file
raw_df = pd.read_csv(
    os.path.join(runs_dir, 'old_inputs', '2021_02_25_parcels_geography.csv'),
    usecols = ['PARCEL_ID', 'juris', 
               'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
               'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat',
               'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
               'eir_exp2020_id', 'eirzoningmodcat'])

# juris-county crosswalk
juris_county_df = pd.read_csv(
    os.path.join(viz_dir, 'crosswalks', 'juris_county_id.csv'),
    usecols = ['juris_name_full', 'county_name', 'baus_output_juris_name', 'shapefile_juris_name'])
juris_county_df.rename(columns={'juris_name_full': 'juris'}, inplace=True)

# parcel spatial file
p10_geo_file = os.path.join(r'M:\Data\GIS layers\UrbanSim smelt\p10_EIR\p10_parcels.shp')
p10_geo = gpd.read_file(p10_geo_file)
p10_geo = p10_geo[['PARCEL_ID', 'geometry']]
p10_geo['PARCEL_ID'] = p10_geo['PARCEL_ID'].apply(lambda x: int(round(x)))

In [4]:
for i in ['juris', 
               'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
               'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat',
               'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
               'eir_exp2020_id', 'eirzoningmodcat']:
    raw_df[i].fillna('NA', inplace=True)

In [5]:
# attach county name
p10_df = raw_df.merge(juris_county_df, on='juris', how='left')

# add geometry to parcels
p10_geographies = pd.merge(
    p10_geo[['PARCEL_ID', 'geometry']],
    p10_df,
    on='PARCEL_ID', how='left')

In [6]:
print(list(p10_geographies))

['PARCEL_ID', 'geometry', 'juris', 'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id', 'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat', 'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id', 'eir_exp2020_id', 'eirzoningmodcat', 'county_name', 'baus_output_juris_name', 'shapefile_juris_name']


#### zoningmodcat spatial file

In [7]:
%%time
# scenario 24, 25: zoning policy based on "fbpzoningmodcat"
p10_fbpzoningmodcat = p10_geographies[[
    'PARCEL_ID', 'geometry', 'juris', 'county_name', 'fbpzoningmodcat']]

fbpzoningmodcat_gdf = p10_fbpzoningmodcat.dissolve(by='fbpzoningmodcat')

fbpzoningmodcat_gdf = fbpzoningmodcat_gdf.reset_index()[['fbpzoningmodcat', 'geometry']]
fbpzoningmodcat_gdf.columns = ['fbpzmodscat', 'geometry']

CPU times: total: 6min 51s
Wall time: 6min 51s


In [8]:
fbpzoningmodcat_gdf.fbpzmodscat.unique()

array(['alamedaGGNADISNAinNA', 'alamedaGGNADISNAinsfd',
       'alamedaGGNAHRADISNAinNA', ..., 'yountvilleNANAHRADISNAinNA',
       'yountvilleNANAHRADISNAinsfd', 'yountvilleNANAHRADISNAoutNA'],
      dtype=object)

In [9]:
fbpzoningmodcat_gdf.shape[0]

2970

In [10]:
fbpzoningmodcat_gdf.fbpzmodscat.nunique()

2970

In [11]:
fbpzoningmodcat_gdf.to_file(os.path.join(viz_spatial_dir, 'fbpzoningmodcat_gdf.shp'))

C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_13632\3937058702.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fbpzoningmodcat_gdf.to_file(os.path.join(viz_spatial_dir, 'fbpzoningmodcat_gdf.shp'))


In [12]:
%%time
# scenario 28: zoning policy based on "eirzoningmodcat"
p10_eirzoningmodcat = p10_geographies[[
    'PARCEL_ID', 'geometry', 'juris', 'county_name', 'eirzoningmodcat']]

eirzoningmodcat_gdf = p10_eirzoningmodcat.dissolve(by='eirzoningmodcat')
eirzoningmodcat_gdf = eirzoningmodcat_gdf.reset_index()[['eirzoningmodcat', 'geometry']]
eirzoningmodcat_gdf.columns = ['eirzmodscat', 'geometry']

CPU times: total: 6min 3s
Wall time: 6min 3s


In [13]:
eirzoningmodcat_gdf.shape[0]

3535

In [14]:
eirzoningmodcat_gdf.eirzmodscat.nunique()

3535

In [15]:
eirzoningmodcat_gdf.eirzmodscat.unique()

array(['alamedaGGNADISNANAinNA', 'alamedaGGNADISNANAinres',
       'alamedaGGNAHRADISNANAinNA', ..., 'yountvilleNANAHRADISNANAinNA',
       'yountvilleNANAHRADISNANAinres', 'yountvilleNANAHRADISNANAoutNA'],
      dtype=object)

In [16]:
eirzoningmodcat_gdf.to_file(os.path.join(viz_spatial_dir, 'eirzoningmodcat_gdf.shp'))

C:\Users\mtcpb\AppData\Local\Temp\2\ipykernel_13632\4174187374.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  eirzoningmodcat_gdf.to_file(os.path.join(viz_spatial_dir, 'eirzoningmodcat_gdf.shp'))


In [17]:
print(list(eirzoningmodcat_gdf))

['eirzmodscat', 'geometry']


#### fbpchcat spatial file (for inclusionary zoning policy)

In [18]:
%%time
# scenario 24, 28: inclusionary zoning based on 'fbpchcat'
p10_fbpchcat = p10_geographies[['PARCEL_ID', 'geometry', 'fbpchcat']]
fbpchcat_gdf = p10_fbpchcat.dissolve(by='fbpchcat')
fbpchcat_gdf = fbpchcat_gdf.reset_index()[['fbpchcat', 'geometry']]
fbpchcat_gdf.columns = ['fbpchcat', 'geometry']

CPU times: total: 12min 52s
Wall time: 12min 52s


In [19]:
fbpchcat_gdf.to_file(os.path.join(viz_spatial_dir, 'fbpchcat_gdf.shp'))

In [20]:
# scenario 25: jurisdiction, using existing juris shapefile

#### non-spatial table with key growth geographies tagging

In [21]:
# FBP growth geographies

# parcel-level
parcel_geo_fbp = p10_geographies[['PARCEL_ID', 'juris', 'baus_output_juris_name', 'shapefile_juris_name', 'county_name',
                                  'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
                                  'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat']]

In [ ]:
p10_fbpzoningmodcat

In [23]:
geo_fbp_attrs = parcel_geo_fbp[['juris', 'shapefile_juris_name',
                                'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id',
                                'fbp_exp2020_id', 'fbpzoningmodcat']].drop_duplicates()
print(geo_fbp_attrs.shape[0])

fbpzoningmodcat_attrs = fbpzoningmodcat_gdf.merge(geo_fbp_attrs, 
                                                         left_on='fbpzmodscat',
                                                         right_on='fbpzoningmodcat',
                                                         how='outer')
print(fbpzoningmodcat_attrs.shape[0])

fbpzoningmodcat_attrs.drop(['fbpzoningmodcat', 'geometry', 'juris'], axis=1, inplace=True)

fbpzoningmodcat_attrs.rename(columns = {'fbp_gg_id': 'gg_id',
                                        'fbp_tra_id': 'tra_id',
                                        'fbp_sesit_id': 'sesit_id',
                                        'fbp_ppa_id': 'ppa_id',
                                        'fbp_exp2020_id': 'exp2020_id'}, inplace=True)

2970
2970


In [24]:
parcel_geo_eir = p10_geographies[['PARCEL_ID', 'juris', 'baus_output_juris_name', 'shapefile_juris_name', 'county_name',
                                  'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
                                  'eir_exp2020_id', 'eirzoningmodcat']]

In [25]:
geo_eir_attrs = parcel_geo_eir[['juris', 'shapefile_juris_name',
                                'eir_gg_id', 'eir_tra_id', 'eir_sesit_id', 'eir_coc_id', 'eir_ppa_id',
                                  'eir_exp2020_id', 'eirzoningmodcat']].drop_duplicates()
print(geo_eir_attrs.shape[0])

eirzoningmodcat_attrs = eirzoningmodcat_gdf.merge(geo_eir_attrs, 
                                                         left_on='eirzmodscat',
                                                         right_on='eirzoningmodcat',
                                                         how='outer')
print(eirzoningmodcat_attrs.shape[0])

eirzoningmodcat_attrs.drop(['eirzoningmodcat','geometry','juris'], axis=1, inplace=True)

eirzoningmodcat_attrs.rename(columns = {'eir_gg_id': 'gg_id',
                                        'eir_tra_id': 'tra_id',
                                        'eir_sesit_id': 'sesit_id',
                                        'eir_coc_id': 'coc_id',
                                        'eir_ppa_id': 'ppa_id',
                                        'eir_exp2020_id': 'exp2020_id'}, inplace=True)

3535
3535


#### simplify the growth geography categories for easier mapping

In [26]:
# fbpzoningmodcat_attrs['tra_id_cat'] = 'non-TRA'
# fbpzoningmodcat_attrs.loc[fbpzoningmodcat_attrs['tra_id'] != 'NA', 'tra_id_cat'] = 'TRA'

# fbpzoningmodcat_attrs['hra_id_cat'] = 'non-HRA'
# fbpzoningmodcat_attrs.loc[fbpzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS']), 'hra_id_cat'] = 'HRA'

fbpzoningmodcat_attrs['simpler_geo_cat'] = ''
fbpzoningmodcat_attrs.loc[fbpzoningmodcat_attrs['gg_id'] == 'NA', 'simpler_geo_cat'] = 'outside GG'

fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / Transit-Rich and High-Resource'

fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])), 'simpler_geo_cat'] = 'GG / Transit-Rich Outside HRA'


fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / High-Resource Outside TRA'


fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (fbpzoningmodcat_attrs['ppa_id'] == 'ppa'), 'simpler_geo_cat'] = 'GG / Priority Production Area'

fbpzoningmodcat_attrs.loc[(fbpzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (fbpzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (fbpzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (fbpzoningmodcat_attrs['ppa_id'] == 'NA'), 'simpler_geo_cat'] = 'GG / others'

In [27]:
print(fbpzoningmodcat_attrs['simpler_geo_cat'].value_counts())

outside GG                             1838
GG / Transit-Rich Outside HRA           395
GG / Transit-Rich and High-Resource     319
GG / others                             207
GG / High-Resource Outside TRA          206
GG / Priority Production Area             5
Name: simpler_geo_cat, dtype: int64


In [28]:
# eirzoningmodcat_attrs['tra_id_cat'] = 'non-TRA'
# eirzoningmodcat_attrs.loc[eirzoningmodcat_attrs['tra_id'] != 'NA', 'tra_id_cat'] = 'TRA'

# eirzoningmodcat_attrs['hra_id_cat'] = 'non-HRA'
# eirzoningmodcat_attrs.loc[eirzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS']), 'hra_id_cat'] = 'HRA'

eirzoningmodcat_attrs['simpler_geo_cat'] = ''
eirzoningmodcat_attrs.loc[eirzoningmodcat_attrs['gg_id'] == 'NA', 'simpler_geo_cat'] = 'outside GG'

eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / Transit-Rich and High-Resource'

eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] != 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])), 'simpler_geo_cat'] = 'GG / Transit-Rich Outside HRA'


eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['HRA', 'HRADIS'])), 'simpler_geo_cat'] = 'GG / High-Resource Outside TRA'


eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (eirzoningmodcat_attrs['ppa_id'] == 'ppa'), 'simpler_geo_cat'] = 'GG / Priority Production Area'

eirzoningmodcat_attrs.loc[(eirzoningmodcat_attrs['gg_id'] == 'GG') & \
                          (eirzoningmodcat_attrs['tra_id'] == 'NA') & \
                          (eirzoningmodcat_attrs['sesit_id'].isin(['DIS', 'NA'])) & \
                          (eirzoningmodcat_attrs['ppa_id'] == 'NA'), 'simpler_geo_cat'] = 'GG / others'

In [29]:
print(eirzoningmodcat_attrs['simpler_geo_cat'].value_counts())

outside GG                             2159
GG / Transit-Rich Outside HRA           559
GG / Transit-Rich and High-Resource     337
GG / others                             273
GG / High-Resource Outside TRA          202
GG / Priority Production Area             5
Name: simpler_geo_cat, dtype: int64


#### export

In [30]:
fbpzoningmodcat_attrs.to_csv(os.path.join(viz_crosswalks_dir, 'fbpzoningmodcat_attrs.csv'), index=False)

In [31]:
eirzoningmodcat_attrs.to_csv(os.path.join(viz_crosswalks_dir, 'eirzoningmodcat_attrs.csv'), index=False)

In [32]:
fbpzoningmodcat_attrs

,fbpzmodscat,shapefile_juris_name,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,simpler_geo_cat
0,alamedaGGNADISNAinNA,Alameda,GG,NA,DIS,NA,in,GG / others
1,alamedaGGNADISNAinsfd,Alameda,GG,NA,DIS,NA,in,GG / others
2,alamedaGGNAHRADISNAinNA,Alameda,GG,NA,HRADIS,NA,in,GG / High-Resource Outside TRA
3,alamedaGGNAHRADISNAinsfd,Alameda,GG,NA,HRADIS,NA,in,GG / High-Resource Outside TRA
4,alamedaGGNAHRANAinNA,Alameda,GG,NA,HRA,NA,in,GG / High-Resource Outside TRA
...,...,...,...,...,...,...,...,...
2965,yountvilleNANADISNAinsfd,Yountville,NA,NA,DIS,NA,in,outside GG
2966,yountvilleNANADISNAoutNA,Yountville,NA,NA,DIS,NA,out,outside GG
2967,yountvilleNANAHRADISNAinNA,Yountville,NA,NA,HRADIS,NA,in,outside GG
2968,yountvilleNANAHRADISNAinsfd,Yountville,NA,NA,HRADIS,NA,in,outside GG
